# CC1 - IA Exploratoire et Symbolique
## Système de Diagnostic Médical Multi-Contraintes pour le Diabète de Type 2

### 🎯 Objectifs Pédagogiques

Ce notebook vise à vous faire implémenter un système de diagnostic médical intelligent combinant quatre approches algorithmiques complémentaires :

1. **Agent de Diagnostic Rationnel** : Système basé sur des règles cliniques
2. **Algorithme A*** : Recherche informée dans l'espace d'états diagnostiques
3. **Algorithmes Génétiques** : Optimisation évolutionnaire des paramètres
4. **Solveur Z3** : Validation par contraintes des protocoles thérapeutiques

### 📊 Contexte Médical

Le diabète de type 2 nécessite une approche de diagnostic personnalisée qui combine :
- **Analyse multi-dimensionnelle** : Glycémie, HbA1c, symptômes, antécédents
- **Contraintes thérapeutiques** : Protocoles médicaux, interactions médicamenteuses
- **Optimisation personnalisée** : Adaptation des traitements selon le profil patient

### 🛠️ Compétences Évaluées

- Algorithmes de recherche (BFS, DFS, A*)
- Programmation par contraintes (Z3, CSP)
- Algorithmes génétiques et optimisation
- Analyse de complexité et performance
- Application biomédicale (diabète type 2)


In [3]:
# Installation des dépendances requises
%pip install numpy pandas matplotlib seaborn z3-solver

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
# Configuration du notebook
CONFIG = {
    'version': '2.0.0',
    'auteur': 'EPF IA Biomédicale',
    'date': '2025-11-04',
    'duree_estimee': '3 heures',
    'points_total': 20
}

# Librairies de base
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import time
import random
import heapq
from typing import List, Optional, Dict, Tuple

# Librairies spécialisées
try:
    from z3 import *  # Solveur de contraintes
    print("✅ Z3 importé avec succès")
except ImportError:
    print("❌ Z3 non disponible. Installez avec : pip install z3-solver")
    
print(f"📋 Configuration : {CONFIG}")
print("🎯 Objectif : Implémenter un système de diagnostic médical multi-approches")

✅ Z3 importé avec succès
📋 Configuration : {'version': '2.0.0', 'auteur': 'EPF IA Biomédicale', 'date': '2025-11-04', 'duree_estimee': '3 heures', 'points_total': 20}
🎯 Objectif : Implémenter un système de diagnostic médical multi-approches


### 📚 Structure de Données

#### Classe Patient

La classe `Patient` représente un patient pour le système de diagnostic. Elle contient toutes les informations médicales nécessaires pour l'analyse.

#### Protocoles Thérapeutiques

Les protocoles définissent les objectifs et contraintes pour le traitement du diabète de type 2.

In [5]:
from dataclasses import dataclass

@dataclass
class Patient:
    """Représente un patient pour le système de diagnostic"""
    id: int
    nom: str
    age: int
    glycemie_jeun: float
    glycemie_postprandiale: float
    hba1c: float
    symptomes: List[str]
    antecedents: List[str]
    date_consultation: datetime
    pression_arterielle: Optional[float] = None
    imc: Optional[float] = None
    
    def __str__(self):
        return f"Patient({self.id}: {self.nom}, {self.age}ans)"

# Protocoles thérapeutiques de référence
protocoles_diabete_type2 = {
    "objectifs_glycemie": {
        "jeun": [80, 130],
        "postprandial": [120, 180]
    },
    "objectifs_hba1c": {
        "cible": "<7.0%",
        "alerte": ">8.0%"
    },
    "contraintes_traitement": {
        "metformine": {
            "contre_indications": ["insuffisance_rénale_sévère", "acidose_lactique"],
            "dose_max": 3000
        },
        "insuline": {
            "contre_indications": ["hypoglycémie_sévère_non_traitée"],
            "ajustement": "selon_glycemie"
        },
        "statines": {
            "contre_indications": ["maladie_hépatique_active"],
            "surveillance": "transaminases_hepatiques"
        }
    },
    "contraintes_globales": [
        "hba1c < 7.0%",
        "pas_d_hypoglycémie_sévère",
        "surveillance_poids",
        "activité_physique_régulière"
    ]
}

print("✅ Structure de données définie")
print(f"📊 Protocoles chargés : {len(protocoles_diabete_type2)} sections")

✅ Structure de données définie
📊 Protocoles chargés : 4 sections


## 🎯 Partie 1 : Agent de Diagnostic Basique

### Théorie des Agents Intelligents (AIMA Chapitre 1)

Un **agent Intelligent** est un système qui perçoit son environnement et agit rationnellement pour atteindre des objectifs. Dans le contexte médical :

- **Perception** : Données patient (glycémie, symptômes, antécédents)
- **Raisonnement** : Application de règles cliniques
- **Action** : Génération de diagnostics et recommandations

### 📋 Objectifs de cette Partie

1. **Classification du risque** : Normal/Pré-diabète/Diabète Type 2
2. **Analyse des symptômes** : Interprétation clinique
3. **Génération de recommandations** : Conseils personnalisés

### 🔧 Implémentation Requise

Vous devez implémenter la classe `DiagnosticAgent` avec les méthodes suivantes :
- `__init__(self)` : Initialisation avec règles cliniques
- `classifier_risque(self, patient: Patient) -> str` : Classification du risque
- `analyser_symptomes(self, patient: Patient) -> List[str]` : Interprétation des symptômes
- `generer_recommandations(self, patient: Patient, risque: str) -> List[str]` : Recommandations personnalisées

In [6]:
class DiagnosticAgent:
    """Agent de diagnostic médical basé sur des règles cliniques"""
    
    def __init__(self):
        self.regles_diagnostiques = self._charger_regles()
    
    def _charger_regles(self) -> Dict:
        """Charge les règles cliniques pour le diagnostic du diabète"""
        return {
            "normal": {
                "glycemie_jeun": (70, 99),
                "glycemie_postprandiale": (70, 139),
                "hba1c": (0, 5.6)
            },
            "prediabete": {
                "glycemie_jeun": (100, 125),
                "glycemie_postprandiale": (140, 199),
                "hba1c": (5.7, 6.4)
            },
            "diabete_type2": {
                "glycemie_jeun": (126, float('inf')),
                "glycemie_postprandiale": (200, float('inf')),
                "hba1c": (6.5, float('inf'))
            }
        }
    
    def classifier_risque(self, patient: Patient) -> str:
        """Classifie le risque diabétique du patient"""
        score_normal = 0
        score_prediabete = 0
        score_diabete = 0
        
        regles = self.regles_diagnostiques
        
        # Évaluation glycémie à jeun
        if regles["normal"]["glycemie_jeun"][0] <= patient.glycemie_jeun <= regles["normal"]["glycemie_jeun"][1]:
            score_normal += 1
        elif regles["prediabete"]["glycemie_jeun"][0] <= patient.glycemie_jeun <= regles["prediabete"]["glycemie_jeun"][1]:
            score_prediabete += 1
        else:
            score_diabete += 1
        
        # Évaluation glycémie post-prandiale
        if regles["normal"]["glycemie_postprandiale"][0] <= patient.glycemie_postprandiale <= regles["normal"]["glycemie_postprandiale"][1]:
            score_normal += 1
        elif regles["prediabete"]["glycemie_postprandiale"][0] <= patient.glycemie_postprandiale <= regles["prediabete"]["glycemie_postprandiale"][1]:
            score_prediabete += 1
        else:
            score_diabete += 1
        
        # Évaluation HbA1c
        if regles["normal"]["hba1c"][0] <= patient.hba1c <= regles["normal"]["hba1c"][1]:
            score_normal += 1
        elif regles["prediabete"]["hba1c"][0] <= patient.hba1c <= regles["prediabete"]["hba1c"][1]:
            score_prediabete += 1
        else:
            score_diabete += 1
        
        # Détermination du risque
        if score_normal >= 2:
            return "Normal"
        elif score_prediabete >= 2:
            return "Pré-diabète"
        else:
            return "Diabète Type 2"
    
    def analyser_symptomes(self, patient: Patient) -> List[str]:
        """Analyse les symptômes du patient"""
        symptomes_diabete = {
            "polyurie": "Augmentation de la fréquence urinaire",
            "polydipsie": "Soif intense et fréquente",
            "polyphagie": "Faim excessive malgré l'alimentation",
            "fatigue": "Fatigue chronique et faiblesse",
            "perte_poids": "Perte de poids inexpliquée",
            "vision_troublee": "Vision floue ou troublee",
            "cicatrisation_lente": "Cicatrisation retardée des plaies",
            "infections_recurrentes": "Infections fréquentes (urinaires, cutanées)"
        }
        
        interpretations = []
        for symptome in patient.symptomes:
            if symptome in symptomes_diabete:
                interpretations.append(f"{symptome}: {symptomes_diabete[symptome]}")
        
        return interpretations
    
    def generer_recommandations(self, patient: Patient, risque: str) -> List[str]:
        """Génère des recommandations basées sur le risque et le profil"""
        recommandations = []
        
        if risque == "Normal":
            recommandations.extend([
                "Maintenir un mode de vie sain",
                "Surveillance annuelle de la glycémie",
                "Activité physique régulière (150 min/semaine)"
            ])
        elif risque == "Pré-diabète":
            recommandations.extend([
                "Perte de poids si surpoids (5-10% du poids)",
                "Activité physique modérée (30 min/jour, 5j/semaine)",
                "Alimentation équilibrée, riche en fibres",
                "Surveillance trimestrielle de la glycémie",
                "Considérer metformine si facteurs de risque élevés"
            ])
        else:  # Diabète Type 2
            recommandations.extend([
                "Consultation spécialisée obligatoire",
                "Traitement médicamenteux immédiat",
                "Autosurveillance glycémique quotidienne",
                "Éducation thérapeutique complète",
                "Adaptation du mode de vie (alimentation, activité)"
            ])
        
        # Recommandations personnalisées selon le profil
        if patient.age > 65:
            recommandations.append("Adaptation des traitements à l'âge avancé")
        
        if patient.imc and patient.imc > 30:
            recommandations.append("Perte de poids prioritaire (objectif: IMC < 25)")
        
        if "hypertension" in patient.antecedents:
            recommandations.append("Surveillance tensionnelle stricte")
        
        return recommandations

# Test de la classe
print("🤖 Création de l'agent de diagnostic...")
agent = DiagnosticAgent()
print("✅ Agent initialisé avec succès")
print(f"📋 Règles chargées : {len(agent.regles_diagnostiques)} catégories")

🤖 Création de l'agent de diagnostic...
✅ Agent initialisé avec succès
📋 Règles chargées : 3 catégories


### 📝 Exemples d'Utilisation Attendus

Voici des exemples de ce que votre agent devrait produire :

#### Cas Normal
```python
patient_normal = Patient(1, "Alice", 35, 85.0, 120.0, 5.2, [], "aucun", datetime.now())
risque = agent.classifier_risque(patient_normal)  # Devrait retourner "Normal"
recommandations = agent.generer_recommandations(patient_normal, "Normal")
# Devrait inclure : "Maintenir un mode de vie sain"
```

#### Cas Pré-diabète
```python
patient_prediabete = Patient(2, "Bob", 48, 115.0, 155.0, 6.0, ["fatigue"], "surpoids", datetime.now())
risque = agent.classifier_risque(patient_prediabete)  # Devrait retourner "Pré-diabète"
recommandations = agent.generer_recommandations(patient_prediabete, "Pré-diabète")
# Devrait inclure : "Perte de poids si surpoids"
```

#### Cas Diabète Type 2
```python
patient_diabete = Patient(3, "Claire", 62, 140.0, 210.0, 7.8, ["polyurie", "soif"], "hypertension", datetime.now())
risque = agent.classifier_risque(patient_diabete)  # Devrait retourner "Diabète Type 2"
recommandations = agent.generer_recommandations(patient_diabete, "Diabète Type 2")
# Devrait inclure : "Consultation spécialisée obligatoire"
```

In [7]:
def tester_agent_diagnostic():
    """Teste l'agent de diagnostic avec des cas connus"""
    agent = DiagnosticAgent()
    
    # Cas test 1 : Patient normal
    patient_normal = Patient(
        1, "Alice", 35, 85.0, 120.0, 5.2, [], 
        ["aucun"], datetime.now()
    )
    
    # Cas test 2 : Patient pré-diabétique
    patient_prediabete = Patient(
        2, "Bob", 48, 115.0, 155.0, 6.0, ["fatigue"], 
        ["surpoids"], datetime.now()
    )
    
    # Cas test 3 : Patient diabétique
    patient_diabete = Patient(
        3, "Claire", 62, 140.0, 210.0, 7.8, ["polyurie", "soif intense"], 
        ["hypertension"], datetime.now()
    )
    
    patients_test = [patient_normal, patient_prediabete, patient_diabete]
    
    print("\n=== Tests de l'Agent de Diagnostic ===")
    for patient in patients_test:
        risque = agent.classifier_risque(patient)
        symptomes = agent.analyser_symptomes(patient)
        recommandations = agent.generer_recommandations(patient, risque)
        
        print(f"\n--- Patient: {patient.nom} ---")
        print(f"📊 Âge: {patient.age} ans")
        print(f"📈 Glycémie jeun: {patient.glycemie_jeun} mg/dL")
        print(f"📈 Glycémie post: {patient.glycemie_postprandiale} mg/dL")
        print(f"📈 HbA1c: {patient.hba1c}%")
        print(f"⚠️ Risque classifié: {risque}")
        print(f"🔍 Symptômes analysés: {len(symptomes)}")
        if symptomes:
            for s in symptomes:
                print(f"   - {s}")
        print(f"💡 Recommandations ({len(recommandations)}):")
        for i, rec in enumerate(recommandations[:3], 1):
            print(f"   {i}. {rec}")

# Exécution des tests
tester_agent_diagnostic()
print("\n✅ Tests de l'agent de diagnostic terminés")


=== Tests de l'Agent de Diagnostic ===

--- Patient: Alice ---
📊 Âge: 35 ans
📈 Glycémie jeun: 85.0 mg/dL
📈 Glycémie post: 120.0 mg/dL
📈 HbA1c: 5.2%
⚠️ Risque classifié: Normal
🔍 Symptômes analysés: 0
💡 Recommandations (3):
   1. Maintenir un mode de vie sain
   2. Surveillance annuelle de la glycémie
   3. Activité physique régulière (150 min/semaine)

--- Patient: Bob ---
📊 Âge: 48 ans
📈 Glycémie jeun: 115.0 mg/dL
📈 Glycémie post: 155.0 mg/dL
📈 HbA1c: 6.0%
⚠️ Risque classifié: Pré-diabète
🔍 Symptômes analysés: 1
   - fatigue: Fatigue chronique et faiblesse
💡 Recommandations (5):
   1. Perte de poids si surpoids (5-10% du poids)
   2. Activité physique modérée (30 min/jour, 5j/semaine)
   3. Alimentation équilibrée, riche en fibres

--- Patient: Claire ---
📊 Âge: 62 ans
📈 Glycémie jeun: 140.0 mg/dL
📈 Glycémie post: 210.0 mg/dL
📈 HbA1c: 7.8%
⚠️ Risque classifié: Diabète Type 2
🔍 Symptômes analysés: 1
   - polyurie: Augmentation de la fréquence urinaire
💡 Recommandations (6):
   1. Cons

## 🎯 Partie 2 : Algorithme A* (Recherche Informée)

### Théorie Recherche Informée (AIMA Chapitres 3-4)

L'algorithme **A\*** (A-star) est un algorithme de recherche informée qui garantit l'optimalité si l'heuristique est admissible. Dans le contexte médical :

- **État** : Représentation d'une hypothèse diagnostique
- **Transition** : Passage d'une hypothèse à une autre
- **Heuristique** : Estimation du coût restant vers l'objectif
- **Optimalité** : Garantie de trouver le meilleur chemin

### 📋 Objectifs de cette Partie

1. **Définir l'espace d'états** : Représentation des hypothèses
2. **Implémenter l'heuristique** : Fonction d'évaluation médicale
3. **Algorithme A\*** : Recherche du diagnostic optimal
4. **Tests de performance** : Mesures temporelles et mémoire

### 🔧 Implémentation Requise

Vous devez implémenter les classes suivantes :
- `EtatDiagnostic` : Représentation d'un état dans le processus de diagnostic
- `AStarDiagnostic` : Algorithme A* pour la recherche diagnostique optimale

In [8]:
class EtatDiagnostic:
    """Représente un état dans le processus de diagnostic"""
    
    def __init__(self, patient: Patient, hypotheses: List[str], niveau_confiance: float):
        self.patient = patient
        self.hypotheses = hypotheses.copy()
        self.niveau_confiance = niveau_confiance
        self.cout = 0.0
    
    def __lt__(self, other):
        """Comparaison pour la file de priorité (min-heap)"""
        return self.cout < other.cout
    
    def __eq__(self, other):
        """Égalité des états"""
        return (self.patient.id == other.patient.id and 
                set(self.hypotheses) == set(other.hypotheses) and
                abs(self.niveau_confiance - other.niveau_confiance) < 0.01)
    
    def __hash__(self):
        """Hash pour les ensembles d'états visités"""
        return hash((self.patient.id, tuple(sorted(self.hypotheses)), 
                   round(self.niveau_confiance, 2)))

class AStarDiagnostic:
    """Algorithme A* pour la recherche diagnostique optimale"""
    
    def __init__(self):
        self.hypotheses_possibles = [
            "diabete_type1", "diabete_type2", "prediabete", "intolerance_glucose",
            "syndrome_metabolique", "diabete_gestationnel", "autre_pathologie"
        ]
        self.poids_evidences = {
            "glycemie_jeun": 0.3,
            "glycemie_postprandiale": 0.25,
            "hba1c": 0.2,
            "symptomes": 0.15,
            "antecedents": 0.1
        }
    
    def heuristique_medical(self, etat: EtatDiagnostic) -> float:
        """Heuristique basée sur les critères cliniques médicaux"""
        patient = etat.patient
        
        # Score de cohérence avec les données patient
        score_evidence = 0.0
        
        # Évidence glycémique
        if patient.glycemie_jeun > 126:
            score_evidence += self.poids_evidences["glycemie_jeun"] * 0.8
        elif patient.glycemie_jeun > 100:
            score_evidence += self.poids_evidences["glycemie_jeun"] * 0.4
        
        if patient.glycemie_postprandiale > 200:
            score_evidence += self.poids_evidences["glycemie_postprandiale"] * 0.8
        elif patient.glycemie_postprandiale > 140:
            score_evidence += self.poids_evidences["glycemie_postprandiale"] * 0.4
        
        # Évidence HbA1c
        if patient.hba1c > 7.0:
            score_evidence += self.poids_evidences["hba1c"] * 0.9
        elif patient.hba1c > 6.0:
            score_evidence += self.poids_evidences["hba1c"] * 0.5
        
        # Évidence symptômes
        symptomes_diabete = ["polyurie", "polydipsie", "polyphagie", "fatigue"]
        score_symptomes = sum(1 for s in patient.symptomes if s in symptomes_diabete)
        if score_symptomes > 0:
            score_evidence += self.poids_evidences["symptomes"] * (score_symptomes / len(symptomes_diabete))
        
        # Pénalité pour hypothèses multiples
        penalite_hypotheses = len(etat.hypotheses) * 0.1
        
        # Heuristique : maximiser la confiance en minimisant les hypothèses
        heuristique = 1.0 - score_evidence + penalite_hypotheses
        
        return heuristique
    
    def cout_transition(self, etat1: EtatDiagnostic, etat2: EtatDiagnostic) -> float:
        """Calcule le coût de transition entre deux états"""
        # Coût basé sur le changement de confiance
        cout_confiance = abs(etat2.niveau_confiance - etat1.niveau_confiance)
        
        # Coût basé sur le nombre d'hypothèses
        cout_hypotheses = abs(len(etat2.hypotheses) - len(etat1.hypotheses)) * 0.1
        
        return cout_confiance + cout_hypotheses
    
    def rechercher_diagnostic_optimal(self, patient: Patient) -> EtatDiagnostic:
        """Recherche le diagnostic optimal avec A*"""
        # État initial : hypothèses larges, confiance faible
        etat_initial = EtatDiagnostic(
            patient, 
            self.hypotheses_possibles.copy(), 
            0.1
        )
        
        # File de priorité pour A* (min-heap)
        file_priorite = []
        heapq.heappush(file_priorite, (0, etat_initial))
        
        # Ensembles des états visités et en cours
        visites = set()
        couts = {etat_initial: 0}
        
        iterations = 0
        max_iterations = 1000
        
        while file_priorite and iterations < max_iterations:
            iterations += 1
            
            # Extraire l'état avec le coût le plus faible
            cout_actuel, etat_actuel = heapq.heappop(file_priorite)
            
            # Vérifier si on a atteint un état final
            if self._est_etat_final(etat_actuel):
                print(f"✓ A* convergé en {iterations} itérations")
                return etat_actuel
            
            # Marquer comme visité
            if etat_actuel in visites:
                continue
            visites.add(etat_actuel)
            
            # Générer les états voisins
            voisins = self._generer_voisins(etat_actuel)
            
            for voisin in voisins:
                nouveau_cout = couts[etat_actuel] + self.cout_transition(etat_actuel, voisin)
                
                if voisin not in couts or nouveau_cout < couts[voisin]:
                    couts[voisin] = nouveau_cout
                    cout_total = nouveau_cout + self.heuristique_medical(voisin)
                    heapq.heappush(file_priorite, (cout_total, voisin))
        
        print(f"⚠ A* n'a pas convergé après {iterations} itérations")
        return etat_actuel
    
    def _est_etat_final(self, etat: EtatDiagnostic) -> bool:
        """Vérifie si l'état est un état final (diagnostic convergé)"""
        return (len(etat.hypotheses) <= 2 and 
                etat.niveau_confiance >= 0.8)
    
    def _generer_voisins(self, etat: EtatDiagnostic) -> List[EtatDiagnostic]:
        """Génère les états voisins possibles"""
        voisins = []
        
        # Stratégie 1 : Éliminer une hypothèse
        for i, hypothese in enumerate(etat.hypotheses):
            nouvelles_hypotheses = etat.hypotheses[:i] + etat.hypotheses[i+1:]
            nouvelle_confiance = min(0.9, etat.niveau_confiance + 0.1)
            
            voisin = EtatDiagnostic(
                etat.patient,
                nouvelles_hypotheses,
                nouvelle_confiance
            )
            voisin.cout = etat.cout + 0.1
            voisins.append(voisin)
        
        # Stratégie 2 : Augmenter la confiance
        nouvelle_confiance = min(0.95, etat.niveau_confiance + 0.15)
        voisin_confiance = EtatDiagnostic(
            etat.patient,
            etat.hypotheses.copy(),
            nouvelle_confiance
        )
        voisin_confiance.cout = etat.cout + 0.05
        voisins.append(voisin_confiance)
        
        return voisins

print("🔍 Création de l'algorithme A*...")
astar = AStarDiagnostic()
print("✅ Algorithme A* initialisé")
print(f"📊 Hypothèses possibles : {len(astar.hypotheses_possibles)}")

🔍 Création de l'algorithme A*...
✅ Algorithme A* initialisé
📊 Hypothèses possibles : 7


### 🧠 Explication Heuristique Médicale

L'heuristique médicale doit évaluer la pertinence d'une hypothèse diagnostique :

- **Score d'évidence** : Cohérence avec les données patient
- **Pénalité hypothèses** : Moins il y a d'hypothèses, mieux c'est
- **Confiance** : Niveau de certitude du diagnostic

### 📊 Exemples de Parcours d'Espace d'États

Le parcours typique pour un patient complexe :
1. **État initial** : Hypothèses larges, confiance faible
2. **Élimination progressive** : Réduction des hypothèses
3. **Augmentation confiance** : Validation des hypothèses restantes
4. **État final** : 1-2 hypothèses, confiance élevée

In [9]:
def tester_performance_astar():
    """Teste les performances de l'algorithme A*"""
    astar_test = AStarDiagnostic()
    
    # Patients de test avec complexité croissante
    patients_test = [
        Patient(1, "Simple", 45, 110.0, 160.0, 6.8, ["fatigue"], 
               ["aucun"], datetime.now()),
        
        Patient(2, "Moyen", 58, 140.0, 200.0, 8.2, ["polyurie", "soif"], 
               ["hypertension"], datetime.now()),
        
        Patient(3, "Complexe", 67, 130.0, 180.0, 7.5, 
               ["fatigue", "vision_troublee", "engourdissement"], 
               ["néphropathie"], datetime.now(), 140.0, 28.5)
    ]
    
    import time
    
    print("\n=== Test de Performance A* ===")
    for i, patient in enumerate(patients_test):
        complexite = ["Simple", "Moyen", "Complexe"][i]
        print(f"\n--- Patient: {patient.nom} (Complexité: {complexite}) ---")
        
        temps_debut = time.time()
        resultat = astar_test.rechercher_diagnostic_optimal(patient)
        temps_fin = time.time()
        
        print(f"⏱️ Temps d'exécution: {(temps_fin - temps_debut)*1000:.2f} ms")
        print(f"🎯 Hypothèses finales: {', '.join(resultat.hypotheses)}")
        print(f"📊 Niveau de confiance: {resultat.niveau_confiance:.3f}")
        print(f"💰 Coût total: {resultat.cout:.3f}")

# Exécution des tests
tester_performance_astar()
print("\n✅ Tests de performance A* terminés")


=== Test de Performance A* ===

--- Patient: Simple (Complexité: Simple) ---
✓ A* convergé en 511 itérations
⏱️ Temps d'exécution: 9.75 ms
🎯 Hypothèses finales: diabete_type2
📊 Niveau de confiance: 0.850
💰 Coût total: 0.650

--- Patient: Moyen (Complexité: Moyen) ---
✓ A* convergé en 511 itérations
⏱️ Temps d'exécution: 9.31 ms
🎯 Hypothèses finales: syndrome_metabolique
📊 Niveau de confiance: 0.850
💰 Coût total: 0.650

--- Patient: Complexe (Complexité: Complexe) ---
✓ A* convergé en 511 itérations
⏱️ Temps d'exécution: 9.34 ms
🎯 Hypothèses finales: syndrome_metabolique
📊 Niveau de confiance: 0.850
💰 Coût total: 0.650

✅ Tests de performance A* terminés


## 🎯 Partie 3 : Algorithmes Génétiques

### Théorie Optimisation Évolutionnaire

Les **algorithmes génétiques** s'inspirent de l'évolution naturelle pour optimiser des solutions :

- **Population** : Ensemble de solutions candidates
- **Sélection** : Choix des meilleurs individus
- **Croisement** : Combinaison de solutions parentes
- **Mutation** : Modifications aléatoires
- **Évolution** : Itérations vers l'optimum

### 📋 Objectifs de cette Partie

1. **Chromosomes** : Représentation des paramètres diagnostiques
2. **Fitness médicale** : Fonction d'évaluation clinique
3. **Évolution** : Algorithme évolutionnaire complet
4. **Tests de convergence** : Suivi de l'optimisation

### 🔧 Implémentation Requise

Vous devez implémenter :
- `ChromosomeDiagnostic` : Représentation des paramètres diagnostiques
- `AlgorithmeGenetiqueDiagnostic` : Algorithme évolutionnaire pour l'optimisation

In [10]:
class ChromosomeDiagnostic:
    """Chromosome représentant les paramètres diagnostiques"""
    
    def __init__(self, genes: Optional[List[float]] = None):
        if genes is None:
            # Gènes : [seuil_glycemie_jeun, seuil_glycemie_post, seuil_hba1c, 
            #          poids_symptomes, poids_antecedents, facteur_age]
            self.genes = [
                random.uniform(90, 140),    # seuil_glycemie_jeun
                random.uniform(120, 200),   # seuil_glycemie_post
                random.uniform(5.5, 8.0),    # seuil_hba1c
                random.uniform(0.1, 0.3),     # poids_symptomes
                random.uniform(0.05, 0.2),   # poids_antecedents
                random.uniform(0.8, 1.2)      # facteur_age
            ]
        else:
            self.genes = genes.copy()
        
        self.fitness = 0.0
    
    def crossover(self, autre: 'ChromosomeDiagnostic') -> Tuple:
        """Croisement avec un autre chromosome"""
        point_croisement = random.randint(1, len(self.genes) - 1)
        
        genes_enfant1 = self.genes[:point_croisement] + autre.genes[point_croisement:]
        genes_enfant2 = autre.genes[:point_croisement] + self.genes[point_croisement:]
        
        return ChromosomeDiagnostic(genes_enfant1), ChromosomeDiagnostic(genes_enfant2)
    
    def mutation(self, taux_mutation: float = 0.1) -> None:
        """Mutation aléatoire des gènes"""
        for i in range(len(self.genes)):
            if random.random() < taux_mutation:
                # Mutation adaptative selon le gène
                if i < 3:  # Seuils glycémiques
                    amplitude = self.genes[i] * 0.1
                    self.genes[i] += random.uniform(-amplitude, amplitude)
                    if i == 0:  # glycemie_jeun
                        self.genes[i] = max(70, min(250, self.genes[i]))
                    elif i == 1:  # glycemie_post
                        self.genes[i] = max(100, min(300, self.genes[i]))
                    elif i == 2:  # hba1c
                        amplitude = 0.5
                        self.genes[i] += random.uniform(-amplitude, amplitude)
                        self.genes[i] = max(4.0, min(12.0, self.genes[i]))
                else:  # Poids et facteurs
                    amplitude = 0.2
                    self.genes[i] += random.uniform(-amplitude, amplitude)
                    self.genes[i] = max(0.0, min(2.0, self.genes[i]))

class AlgorithmeGenetiqueDiagnostic:
    """Algorithme génétique pour optimiser les paramètres diagnostiques"""
    
    def __init__(self, taille_population: int = 50):
        self.taille_population = taille_population
        self.taux_mutation = 0.1
        self.taux_croisement = 0.8
        self.elitisme = 0.2  # Top 20% conservés
        self.generations_max = 100
        self.patients_reference = self._charger_patients_reference()
    
    def _charger_patients_reference(self) -> List[Patient]:
        """Charge les patients de référence pour l'évaluation"""
        return [
            Patient(1, "Ref1", 45, 110.0, 160.0, 6.8, ["fatigue"], 
                   ["aucun"], datetime.now()),
            Patient(2, "Ref2", 58, 140.0, 200.0, 8.2, ["polyurie", "soif"], 
                   ["hypertension"], datetime.now()),
            Patient(3, "Ref3", 67, 130.0, 180.0, 7.5, 
                   ["fatigue", "vision_troublee"], 
                   ["néphropathie"], datetime.now()),
            Patient(4, "Ref4", 35, 90.0, 150.0, 5.5, ["polyurie"], 
                   ["surpoids"], datetime.now()),
            Patient(5, "Ref5", 72, 150.0, 220.0, 9.1, 
                   ["engourdissement", "maux_pieds"], 
                   ["cardiopathie"], datetime.now())
        ]
    
    def fitness_medical(self, chromosome: ChromosomeDiagnostic) -> float:
        """Fonction de fitness basée sur les critères médicaux"""
        seuil_jeun, seuil_post, seuil_hba1c, poids_symptomes, poids_antecedents, facteur_age = chromosome.genes
        
        score_total = 0.0
        patients_corrects = 0
        
        for patient in self.patients_reference:
            score_patient = 0.0
            
            # Évaluation glycémie à jeun
            if patient.glycemie_jeun > seuil_jeun:
                if patient.hba1c > 6.5:  # Vrai positif probable
                    score_patient += 1.0
                else:
                    score_patient -= 0.5  # Faux positif
            else:
                if patient.hba1c < 6.0:  # Vrai négatif probable
                    score_patient += 1.0
                else:
                    score_patient -= 0.5  # Faux négatif
            
            # Évaluation glycémie post-prandiale
            if patient.glycemie_postprandiale > seuil_post:
                if len(patient.symptomes) > 0:  # Symptômes présents
                    score_patient += poids_symptomes * 0.5
                else:
                    score_patient -= 0.3
            else:
                if len(patient.symptomes) == 0:
                    score_patient += 0.5
            
            # Évaluation HbA1c
            if patient.hba1c > seuil_hba1c:
                if "hypertension" in patient.antecedents or "cardiopathie" in patient.antecedents:
                    score_patient += poids_antecedents * 0.3
                else:
                    score_patient += 0.2
            else:
                score_patient += 0.3
            
            # Facteur âge
            if patient.age > 60:
                score_patient += facteur_age * 0.1
            
            score_total += max(0, score_patient)
            if score_patient > 0:
                patients_corrects += 1
        
        # Fitness = score total + bonus pour couverture
        couverture = patients_corrects / len(self.patients_reference)
        fitness = score_total + couverture * 10
        
        # Pénalité pour seuils irréalistes
        if seuil_jeun < 70 or seuil_jeun > 200:
            fitness -= 5
        if seuil_post < 100 or seuil_post > 300:
            fitness -= 5
        if seuil_hba1c < 4.0 or seuil_hba1c > 12.0:
            fitness -= 5
        
        return fitness
    
    def evolution(self) -> ChromosomeDiagnostic:
        """Lance l'évolution génétique"""
        # Initialisation de la population
        population = [ChromosomeDiagnostic() for _ in range(self.taille_population)]
        
        # Évaluation initiale
        for chromosome in population:
            chromosome.fitness = self.fitness_medical(chromosome)
        
        meilleure_solution = max(population, key=lambda c: c.fitness)
        
        print(f"🧬 Début de l'évolution génétique...")
        print(f"📊 Meilleur fitness initial: {meilleure_solution.fitness:.3f}")
        
        for generation in range(self.generations_max):
            nouvelle_population = []
            
            # Élitisme : conserver les meilleurs
            population_triee = sorted(population, key=lambda c: c.fitness, reverse=True)
            elite_size = int(self.taille_population * self.elitisme)
            nouvelle_population.extend(population_triee[:elite_size])
            
            # Génération du reste de la population
            while len(nouvelle_population) < self.taille_population:
                # Sélection par tournoi
                parent1 = self._selection_tournoi(population)
                parent2 = self._selection_tournoi(population)
                
                # Croisement
                if random.random() < self.taux_croisement:
                    enfant1, enfant2 = parent1.crossover(parent2)
                else:
                    enfant1, enfant2 = ChromosomeDiagnostic(parent1.genes.copy()), ChromosomeDiagnostic(parent2.genes.copy())
                
                # Mutation
                enfant1.mutation(self.taux_mutation)
                enfant2.mutation(self.taux_mutation)
                
                # Évaluation
                enfant1.fitness = self.fitness_medical(enfant1)
                enfant2.fitness = self.fitness_medical(enfant2)
                
                nouvelle_population.extend([enfant1, enfant2])
            
            population = nouvelle_population[:self.taille_population]
            
            # Mise à jour de la meilleure solution
            generation_meilleur = max(population, key=lambda c: c.fitness)
            if generation_meilleur.fitness > meilleure_solution.fitness:
                meilleure_solution = generation_meilleur
            
            # Affichage de progression
            if generation % 10 == 0:
                print(f"Génération {generation}: Meilleur fitness = {meilleure_solution.fitness:.3f}")
        
        print(f"✅ Évolution terminée. Meilleur fitness final: {meilleure_solution.fitness:.3f}")
        return meilleure_solution
    
    def _selection_tournoi(self, population: List[ChromosomeDiagnostic]) -> ChromosomeDiagnostic:
        """Sélection par tournoi"""
        taille_tournoi = 3
        participants = random.sample(population, min(taille_tournoi, len(population)))
        return max(participants, key=lambda c: c.fitness)

print("🧬 Création de l'algorithme génétique...")
genetique = AlgorithmeGenetiqueDiagnostic()
print("✅ Algorithme génétique initialisé")
print(f"👥 Patients de référence : {len(genetique.patients_reference)}")

🧬 Création de l'algorithme génétique...
✅ Algorithme génétique initialisé
👥 Patients de référence : 5


### 🏥️ Explication Fitness Médicale

La fonction fitness médicale évalue la qualité d'un chromosome :

- **Précision diagnostique** : Capacité à classifier correctement
- **Cohérence clinique** : Respect des protocoles médicaux
- **Réalisme des seuils** : Valeurs médicalement valides
- **Couverture** : Pourcentage de patients correctement classifiés

### 📊 Tests de Convergence

Le suivi de convergence doit inclure :
- **Historique de fitness** : Évolution de la meilleure solution
- **Graphique** : Visualisation de la convergence
- **Paramètres finaux** : Seuils optimisés obtenus

In [11]:
def tester_convergence_genetique():
    """Teste la convergence de l'algorithme génétique"""
    import matplotlib.pyplot as plt
    
    algorithme = AlgorithmeGenetiqueDiagnostic(taille_population=30)
    algorithme.generations_max = 50  # Réduit pour test rapide
    
    print("\n=== Test de Convergence Génétique ===")
    solution_optimale = algorithme.evolution()
    
    # Afficher les résultats
    print("\n--- Paramètres Optimisés ---")
    noms_genes = ["Seuil Glycémie Jeun (mg/dL)", "Seuil Glycémie Post (mg/dL)", "Seuil HbA1c (%)", 
                  "Poids Symptômes", "Poids Antécédents", "Facteur Âge"]
    
    for nom, valeur in zip(noms_genes, solution_optimale.genes):
        if "Seuil" in nom:
            print(f"  {nom}: {valeur:.1f}")
        else:
            print(f"  {nom}: {valeur:.3f}")
    
    print(f"\n📊 Fitness finale: {solution_optimale.fitness:.3f}")
    
    return solution_optimale

# Exécution du test
resultat = tester_convergence_genetique()
print("\n✅ Tests de convergence génétique terminés")


=== Test de Convergence Génétique ===
🧬 Début de l'évolution génétique...
📊 Meilleur fitness initial: 16.674
Génération 0: Meilleur fitness = 17.002
Génération 10: Meilleur fitness = 18.323
Génération 20: Meilleur fitness = 19.637
Génération 30: Meilleur fitness = 21.550
Génération 40: Meilleur fitness = 21.794
✅ Évolution terminée. Meilleur fitness final: 21.891

--- Paramètres Optimisés ---
  Seuil Glycémie Jeun (mg/dL): 107.8
  Seuil Glycémie Post (mg/dL): 132.9
  Seuil HbA1c (%): 9.5
  Poids Symptômes: 2.000
  Poids Antécédents: 0.305
  Facteur Âge: 1.954

📊 Fitness finale: 21.891

✅ Tests de convergence génétique terminés


## 🎯 Partie 4 : Solveur Z3 (Programmation par Contraintes)

### Théorie Programmation par Contraintes (AIMA Chapitre 6)

La **programmation par Contraintes** (CSP - Constraint Satisfaction Problem) vise à trouver une solution qui satisfait un ensemble de contraintes :

- **Variables** : Inconnues à déterminer
- **Domaines** : Valeurs possibles pour chaque variable
- **Contraintes** : Relations entre variables
- **Solveur** : Algorithme de résolution (Z3)

### 📋 Objectifs de cette Partie

1. **Modélisation CSP** : Traduction des contraintes médicales
2. **Solveur Z3** : Utilisation efficace de Z3
3. **Validation** : Vérification des protocoles
4. **Analyse de faisabilité** : Performance sur plusieurs patients

### 🔧 Implémentation Requise

Vous devez implémenter la classe `Z3ConstraintSolver` avec les méthodes suivantes :
- `definir_variables_contraintes(self, patient: Patient) -> Dict`
- `valider_protocole(self, patient: Patient) -> Dict`
- `analyser_faisabilite(self, patients: List[Patient]) -> Dict`

In [13]:
from z3 import *

class Z3ConstraintSolver:
    """Solveur de contraintes pour validation des protocoles thérapeutiques"""
    
    def __init__(self):
        self.solver = Solver()
        self.protocoles = protocoles_diabete_type2
    
    def definir_variables_contraintes(self, patient: Patient) -> Dict:
        """Définit les variables et contraintes pour un patient"""
        # Variables de décision
        traitement_metformine = Bool('metformine')
        traitement_insuline = Bool('insuline')
        traitement_statines = Bool('statines')
        
        dose_metformine = Real('dose_metformine')
        dose_insuline = Real('dose_insuline')
        objectif_hba1c = Real('objectif_hba1c')
        
        # Contraintes de base
        self.solver.add(objectif_hba1c >= 6.0)  # Objectif minimum
        self.solver.add(objectif_hba1c <= 8.0)  # Objectif réaliste
        
        # Contraintes glycémiques
        self.solver.add(Implies(traitement_metformine, 
                           And(dose_metformine >= 500, dose_metformine <= 3000)))
        self.solver.add(Implies(traitement_insuline, dose_insuline >= 0))
        
        # Contraintes d'âge
        if patient.age > 65:
            self.solver.add(dose_metformine <= 2000)  # Dose réduite pour âgés
        
        # Contraintes de fonction rénale (si données disponibles)
        if "insuffisance_rénale" in patient.antecedents or "insuffisance_rénale_sévère" in patient.antecedents:
            self.solver.add(Not(traitement_metformine))  # Contre-indication
        
        # Contraintes d'IMC
        if patient.imc and patient.imc > 30:
            self.solver.add(objectif_hba1c <= 7.0)  # Objectif plus strict pour obèses
        
        # Contraintes de pression artérielle
        if patient.pression_arterielle and patient.pression_arterielle > 140:
            self.solver.add(traitement_statines)  # Statines recommandées
        
        # Contraintes d'interaction médicamenteuse
        self.solver.add(Implies(And(traitement_metformine, traitement_insuline),
                           dose_metformine <= 2500))  # Réduction si association
        
        # Variables de retour
        return {
            'metformine': traitement_metformine,
            'insuline': traitement_insuline,
            'statines': traitement_statines,
            'dose_metformine': dose_metformine,
            'dose_insuline': dose_insuline,
            'objectif_hba1c': objectif_hba1c
        }
    
    def valider_protocole(self, patient: Patient) -> Dict:
        """Valide le protocole thérapeutique pour un patient"""
        # Réinitialiser le solveur
        self.solver = Solver()
        variables = self.definir_variables_contraintes(patient)
        
        # Tenter de trouver une solution
        if self.solver.check() == sat:
            modele = self.solver.model()
            
            # Extraire les valeurs optimales avec gestion des erreurs
            try:
                metf_val = modele[variables['metformine']]
                insu_val = modele[variables['insuline']]
                stat_val = modele[variables['statines']]
                
                # Conversion sécurisée des booléens
                traitement_metformine = is_true(metf_val) if metf_val is not None else False
                traitement_insuline = is_true(insu_val) if insu_val is not None else False
                traitement_statines = is_true(stat_val) if stat_val is not None else False
                
                # Extraction des doses
                dose_metf = modele[variables['dose_metformine']]
                dose_insu = modele[variables['dose_insuline']]
                obj_hba1c = modele[variables['objectif_hba1c']]
                
                protocole_optimal = {
                    'traitement_metformine': traitement_metformine,
                    'traitement_insuline': traitement_insuline,
                    'traitement_statines': traitement_statines,
                    'dose_metformine': float(dose_metf.as_decimal(2)) if traitement_metformine and dose_metf is not None else 0,
                    'dose_insuline': float(dose_insu.as_decimal(2)) if traitement_insuline and dose_insu is not None else 0,
                    'objectif_hba1c': float(obj_hba1c.as_decimal(2)) if obj_hba1c is not None else 7.0,
                    'valide': True,
                    'contraintes_satisfaites': self._verifier_contraintes(modele, patient, variables)
                }
            except Exception as e:
                protocole_optimal = {
                    'valide': False,
                    'raison': f"Erreur d'extraction: {str(e)}",
                    'contraintes_violees': []
                }
        else:
            protocole_optimal = {
                'valide': False,
                'raison': "Aucune solution trouvée",
                'contraintes_violees': self._identifier_contraintes_violees(patient)
            }
        
        return protocole_optimal
    
    def _verifier_contraintes(self, modele, patient: Patient, variables: Dict) -> List[str]:
        """Vérifie quelles contraintes sont satisfaites"""
        contraintes_satisfaites = []
        
        try:
            # Vérification des contraintes glycémiques
            obj_hba1c = modele[variables['objectif_hba1c']]
            if obj_hba1c is not None:
                objectif_val = float(obj_hba1c.as_decimal(2))
                if 6.0 <= objectif_val <= 8.0:
                    contraintes_satisfaites.append("Objectif HbA1c réaliste")
            
            # Vérification des doses
            if is_true(modele[variables['metformine']]):
                dose = modele[variables['dose_metformine']]
                if dose is not None:
                    dose_val = float(dose.as_decimal(2))
                    if 500 <= dose_val <= 3000:
                        contraintes_satisfaites.append("Dose Metformine appropriée")
            
            # Vérification des contre-indications
            if "insuffisance_rénale" in patient.antecedents:
                if not is_true(modele[variables['metformine']]):
                    contraintes_satisfaites.append("Contre-indication Metformine respectée")
        except Exception:
            pass
        
        return contraintes_satisfaites
    
    def _identifier_contraintes_violees(self, patient: Patient) -> List[str]:
        """Identifie les contraintes potentiellement violées"""
        contraintes_violees = []
        
        # Contraintes d'âge
        if patient.age > 65:
            contraintes_violees.append("Adaptation dose nécessaire pour patient âgé")
        
        # Contraintes d'IMC
        if patient.imc and patient.imc > 35:
            contraintes_violees.append("IMC très élevé - protocole complexe")
        
        # Contraintes de comorbidités
        comorbidites_severes = ["insuffisance_cardiaque", "néphropathie_sévère"]
        if any(com in patient.antecedents for com in comorbidites_severes):
            contraintes_violees.append("Comorbidités sévères - protocole spécialisé requis")
        
        return contraintes_violees
    
    def analyser_faisabilite(self, patients: List[Patient]) -> Dict:
        """Analyse la faisabilit é du système pour plusieurs patients"""
        resultats = {
            'patients_valides': 0,
            'patients_rejetes': 0,
            'contraintes_frequentes': {},
            'temps_moyen_resolution': 0,
            'details_patients': []
        }
        
        import time
        temps_total = 0
        
        for patient in patients:
            temps_debut = time.time()
            protocole = self.valider_protocole(patient)
            temps_fin = time.time()
            
            temps_total += (temps_fin - temps_debut)
            
            if protocole['valide']:
                resultats['patients_valides'] += 1
            else:
                resultats['patients_rejetes'] += 1
            
            # Comptage des contraintes fréquentes
            if not protocole['valide']:
                for contrainte in protocole.get('contraintes_violees', []):
                    resultats['contraintes_frequentes'][contrainte] = \
                        resultats['contraintes_frequentes'].get(contrainte, 0) + 1
            
            resultats['details_patients'].append({
                'patient_id': patient.id,
                'valide': protocole['valide'],
                'protocole': protocole
            })
        
        resultats['temps_moyen_resolution'] = temps_total / len(patients) if patients else 0
        resultats['taux_validation'] = resultats['patients_valides'] / len(patients) if patients else 0
        
        return resultats

print("🔧 Création du solveur Z3...")
solveur = Z3ConstraintSolver()
print("✅ Solveur Z3 initialisé")
print(f"📋 Protocoles chargés : {len(solveur.protocoles)} sections")

🔧 Création du solveur Z3...
✅ Solveur Z3 initialisé
📋 Protocoles chargés : 4 sections


### 🏥️ Explication Modélisation Contraintes Médicales

La modélisation des contraintes médicales doit inclure :

- **Variables de traitement** : Metformine, Insuline, Statines
- **Contraintes de dose** : Limites selon âge, poids, comorbidités
- **Contre-indications** : Exclusions selon antécédents
- **Interactions** : Contraintes entre médicaments
- **Objectifs thérapeutiques** : Cibles HbA1c, glycémie

### 📊 Tests de Validation

Les tests doivent valider :
- **Protocoles complexes** : Patients avec comorbidités
- **Contraintes violées** : Détection des incompatibilités
- **Temps de résolution** : Performance du solveur

In [14]:
def tester_validation_z3():
    """Teste le solveur Z3 avec des cas complexes"""
    solveur_test = Z3ConstraintSolver()
    
    # Cas de test complexes
    patients_complexes = [
        Patient(1, "Cas1", 67, 130.0, 180.0, 7.5, 
               ["fatigue", "vision_troublee"], 
               ["néphropathie", "hypertension"], 
               datetime.now(), 140.0, 28.5),
        
        Patient(2, "Cas2", 72, 150.0, 220.0, 9.1, 
               ["engourdissement", "maux_pieds"], 
               ["cardiopathie"], 
               datetime.now(), 160.0, 32.1),
        
        Patient(3, "Cas3", 55, 125.0, 170.0, 6.9, 
               ["hypoglycémie"], 
               ["hypoglycémie_récurrente"], 
               datetime.now(), 130.0, 27.8)
    ]
    
    print("\n=== Test de Validation Z3 ===")
    
    for patient in patients_complexes:
        print(f"\n--- Patient: {patient.nom} ---")
        print(f"👤 Âge: {patient.age} ans, IMC: {patient.imc:.1f}")
        print(f"📋 Antécédents: {', '.join(patient.antecedents)}")
        
        resultat = solveur_test.valider_protocole(patient)
        
        if resultat['valide']:
            print("✅ Protocole VALIDÉ")
            print(f"   💊 Metformine: {resultat['traitement_metformine']} ({resultat['dose_metformine']:.0f} mg)")
            print(f"   💉 Insuline: {resultat['traitement_insuline']} ({resultat['dose_insuline']:.1f} UI)")
            print(f"   💊 Statines: {resultat['traitement_statines']}")
            print(f"   🎯 Objectif HbA1c: {resultat['objectif_hba1c']:.1f}%")
            print(f"   ✓ Contraintes satisfaites: {len(resultat['contraintes_satisfaites'])}")
            for c in resultat['contraintes_satisfaites']:
                print(f"      - {c}")
        else:
            print("❌ Protocole NON VALIDÉ")
            print(f"   ⚠️ Raison: {resultat.get('raison', 'Inconnue')}")
            if resultat.get('contraintes_violees'):
                print(f"   ⚠️ Contraintes violées:")
                for c in resultat['contraintes_violees']:
                    print(f"      - {c}")
    
    # Analyse globale
    print(f"\n=== Analyse Globale ===")
    faisabilite = solveur_test.analyser_faisabilite(patients_complexes)
    print(f"📊 Taux de validation: {faisabilite['taux_validation']:.1%}")
    print(f"⏱️ Temps moyen de résolution: {faisabilite['temps_moyen_resolution']*1000:.2f} ms")
    if faisabilite['contraintes_frequentes']:
        print(f"⚠️ Contraintes fréquentes:")
        for contrainte, count in faisabilite['contraintes_frequentes'].items():
            print(f"   - {contrainte}: {count} fois")

# Exécution des tests
tester_validation_z3()
print("\n✅ Tests de validation Z3 terminés")


=== Test de Validation Z3 ===

--- Patient: Cas1 ---
👤 Âge: 67 ans, IMC: 28.5
📋 Antécédents: néphropathie, hypertension
✅ Protocole VALIDÉ
   💊 Metformine: False (0 mg)
   💉 Insuline: False (0.0 UI)
   💊 Statines: False
   🎯 Objectif HbA1c: 6.0%
   ✓ Contraintes satisfaites: 1
      - Objectif HbA1c réaliste

--- Patient: Cas2 ---
👤 Âge: 72 ans, IMC: 32.1
📋 Antécédents: cardiopathie
✅ Protocole VALIDÉ
   💊 Metformine: False (0 mg)
   💉 Insuline: False (0.0 UI)
   💊 Statines: True
   🎯 Objectif HbA1c: 6.0%
   ✓ Contraintes satisfaites: 1
      - Objectif HbA1c réaliste

--- Patient: Cas3 ---
👤 Âge: 55 ans, IMC: 27.8
📋 Antécédents: hypoglycémie_récurrente
✅ Protocole VALIDÉ
   💊 Metformine: False (0 mg)
   💉 Insuline: False (0.0 UI)
   💊 Statines: False
   🎯 Objectif HbA1c: 8.0%
   ✓ Contraintes satisfaites: 1
      - Objectif HbA1c réaliste

=== Analyse Globale ===
📊 Taux de validation: 100.0%
⏱️ Temps moyen de résolution: 1.24 ms

✅ Tests de validation Z3 terminés


## 🔄 Pipeline et Tests d'Intégration

### 📋 Objectifs du Pipeline

Le pipeline doit intégrer les 4 approches de manière cohérente :

1. **Chargement des données** : Import depuis CSV
2. **Traitement séquentiel** : Agent → A* → Génétique → Z3
3. **Synthèse des résultats** : Combinaison des approches
4. **Validation croisée** : Vérification de la cohérence

### 🧪 Tests Automatisés

Les tests automatisés doivent valider :
- **Fonctionnalité individuelle** : Chaque approche testée séparément
- **Intégration** : Pipeline complet fonctionnel
- **Performance globale** : Mesures sur l'ensemble du système
- **Robustesse** : Gestion des erreurs et cas limites

In [ ]:
# Chargement des données
def charger_donnees_csv(fichier: str) -> List[Patient]:
    """Charge les données patients depuis un fichier CSV"""
    import pandas as pd
    
    try:
        df = pd.read_csv(fichier)
        patients = []
        
        for _, row in df.iterrows():
            # Traiter les symptômes (séparés par ;)
            symptomes = row['symptomes'].split(';') if isinstance(row['symptomes'], str) and row['symptomes'] != 'aucun' else []
            
            # Traiter les antécédents (séparés par ;)
            antecedents = row['antecedents'].split(';') if isinstance(row['antecedents'], str) else [row['antecedents']]
            
            patient = Patient(
                id=int(row['id']),
                nom=row['nom'],
                age=int(row['age']),
                glycemie_jeun=float(row['glycemie_jeun']),
                glycemie_postprandiale=float(row['glycemie_postprandiale']),
                hba1c=float(row['hba1c']),
                symptomes=symptomes,
                antecedents=antecedents,
                date_consultation=datetime.now(),
                pression_arterielle=float(row['pression_arterielle']) if pd.notna(row.get('pression_arterielle')) else None,
                imc=float(row['imc']) if pd.notna(row.get('imc')) else None
            )
            patients.append(patient)
        
        print(f"✅ {len(patients)} patients chargés depuis {fichier}")
        return patients
    
    except Exception as e:
        print(f"❌ Erreur lors du chargement : {str(e)}")
        return []

# Pipeline principal
def main():
    """Pipeline complet d'intégration des 4 approches"""
    print("\n" + "="*60)
    print("🏥 SYSTÈME DE DIAGNOSTIC MÉDICAL MULTI-CONTRAINTES")
    print("Application: Diabète de Type 2")
    print("="*60)
    
    # Chargement des données
    print("\n📂 Chargement des données patients...")
    patients = charger_donnees_csv("data/patients.csv")
    
    if not patients:
        print("⚠️ Aucun patient chargé, utilisation de données de test")
        patients = [
            Patient(1, "Test1", 45, 110.0, 160.0, 6.8, ["fatigue"], ["aucun"], datetime.now()),
            Patient(2, "Test2", 58, 140.0, 200.0, 8.2, ["polyurie"], ["hypertension"], datetime.now())
        ]
    
    # Initialisation des composants
    print("\n🤖 Initialisation des composants IA...")
    agent = DiagnosticAgent()
    astar_algo = AStarDiagnostic()
    solveur_z3 = Z3ConstraintSolver()
    print("✅ Tous les composants initialisés")
    
    # Traitement pour chaque patient
    print("\n📊 Analyse des patients...")
    for i, patient in enumerate(patients[:3], 1):  # Limité à 3 pour l'exemple
        print(f"\n{'='*60}")
        print(f"Patient {i}/{min(3, len(patients))}: {patient.nom} (ID: {patient.id})")
        print(f"{'='*60}")
        print(f"📋 Âge: {patient.age} ans")
        print(f"📈 Glycémie jeun: {patient.glycemie_jeun} mg/dL")
        print(f"📈 Glycémie post: {patient.glycemie_postprandiale} mg/dL")
        print(f"📈 HbA1c: {patient.hba1c}%")
        
        # Pipeline à 4 étapes
        print("\n🔍 Étape 1: Agent de Diagnostic")
        diagnostic = agent.classifier_risque(patient)
        symptomes = agent.analyser_symptomes(patient)
        recommandations = agent.generer_recommandations(patient, diagnostic)
        print(f"   → Risque: {diagnostic}")
        print(f"   → Symptômes analysés: {len(symptomes)}")
        print(f"   → Recommandations: {recommandations[0] if recommandations else 'Aucune'}")
        
        print("\n🎯 Étape 2: Optimisation A*")
        try:
            optimisation = astar_algo.rechercher_diagnostic_optimal(patient)
            print(f"   → Hypothèses: {', '.join(optimisation.hypotheses[:2])}")
            print(f"   → Confiance: {optimisation.niveau_confiance:.2f}")
        except Exception as e:
            print(f"   ⚠️ Erreur A*: {str(e)}")
        
        print("\n🔧 Étape 3: Validation Z3")
        try:
            protocole = solveur_z3.valider_protocole(patient)
            if protocole['valide']:
                print(f"   ✅ Protocole validé")
                print(f"   → Metformine: {protocole['traitement_metformine']}")
                print(f"   → Insuline: {protocole['traitement_insuline']}")
            else:
                print(f"   ❌ Protocole non validé: {protocole.get('raison')}")
        except Exception as e:
            print(f"   ⚠️ Erreur Z3: {str(e)}")
        
        print("\n--- Synthèse ---")
        print(f"✓ Diagnostic principal: {diagnostic}")
        print(f"✓ Approches validées: Agent + A* + Z3")
        print(f"✓ Recommandation prioritaire: {recommandations[0] if recommandations else 'N/A'}")
    
    print("\n" + "="*60)
    print("✅ Analyse terminée pour tous les patients")
    print("="*60)

# Note: La fonction main() sera exécutée manuellement par l'utilisateur
print("\n💡 Pour exécuter le pipeline complet, appelez: main()")


💡 Pour exécuter le pipeline complet, appelez: main()


In [ ]:
def tests_automatises():
    """Suite de tests automatisés pour validation complète"""
    print("\n" + "="*60)
    print("🧪 SUITE DE TESTS AUTOMATISÉS")
    print("="*60)
    
    tests_resultats = {
        'total': 0,
        'passes': 0,
        'echecs': 0,
        'details': []
    }
    
    # Test 1: Agent de Diagnostic
    print("\n📋 Test 1: Agent de Diagnostic")
    try:
        agent_test = DiagnosticAgent()
        patient_test = Patient(999, "TestAgent", 50, 120.0, 170.0, 7.0, ["fatigue"], ["aucun"], datetime.now())
        risque = agent_test.classifier_risque(patient_test)
        assert risque in ["Normal", "Pré-diabète", "Diabète Type 2"], "Classification invalide"
        print("   ✅ Agent de diagnostic: RÉUSSI")
        tests_resultats['passes'] += 1
        tests_resultats['total'] += 1
    except Exception as e:
        print(f"   ❌ Agent de diagnostic: ÉCHEC - {str(e)}")
        tests_resultats['echecs'] += 1
        tests_resultats['total'] += 1
        tests_resultats['details'].append(('Agent Diagnostic', str(e)))
    
    # Test 2: Performance A*
    print("\n📋 Test 2: Performance A*")
    try:
        import time
        astar_test = AStarDiagnostic()
        patient_test = Patient(999, "TestAstar", 55, 130.0, 180.0, 7.2, ["polyurie"], ["hypertension"], datetime.now())
        
        temps_debut = time.time()
        resultat = astar_test.rechercher_diagnostic_optimal(patient_test)
        temps_fin = time.time()
        
        temps_execution = (temps_fin - temps_debut) * 1000
        assert temps_execution < 5000, f"Temps d'exécution trop long: {temps_execution:.0f}ms"
        assert len(resultat.hypotheses) <= 7, "Trop d'hypothèses"
        print(f"   ✅ Performance A*: RÉUSSI ({temps_execution:.0f}ms)")
        tests_resultats['passes'] += 1
        tests_resultats['total'] += 1
    except Exception as e:
        print(f"   ❌ Performance A*: ÉCHEC - {str(e)}")
        tests_resultats['echecs'] += 1
        tests_resultats['total'] += 1
        tests_resultats['details'].append(('A* Performance', str(e)))
    
    # Test 3: Convergence Génétique
    print("\n📋 Test 3: Convergence Génétique")
    try:
        genetique_test = AlgorithmeGenetiqueDiagnostic(taille_population=20)
        genetique_test.generations_max = 30  # Réduit pour test rapide
        
        solution = genetique_test.evolution()
        assert solution.fitness > 0, "Fitness invalide"
        assert len(solution.genes) == 6, "Nombre de gènes incorrect"
        print(f"   ✅ Convergence génétique: RÉUSSI (fitness={solution.fitness:.2f})")
        tests_resultats['passes'] += 1
        tests_resultats['total'] += 1
    except Exception as e:
        print(f"   ❌ Convergence génétique: ÉCHEC - {str(e)}")
        tests_resultats['echecs'] += 1
        tests_resultats['total'] += 1
        tests_resultats['details'].append(('Génétique Convergence', str(e)))
    
    # Test 4: Validation Z3
    print("\n📋 Test 4: Validation Z3")
    try:
        solveur_test = Z3ConstraintSolver()
        patient_test = Patient(999, "TestZ3", 60, 125.0, 175.0, 7.1,
                             ["fatigue"], ["hypertension"], datetime.now(), 135.0, 26.5)
        
        protocole = solveur_test.valider_protocole(patient_test)
        assert 'valide' in protocole, "Format de protocole invalide"
        print(f"   ✅ Validation Z3: RÉUSSI (valide={protocole['valide']})")
        tests_resultats['passes'] += 1
        tests_resultats['total'] += 1
    except Exception as e:
        print(f"   ❌ Validation Z3: ÉCHEC - {str(e)}")
        tests_resultats['echecs'] += 1
        tests_resultats['total'] += 1
        tests_resultats['details'].append(('Z3 Validation', str(e)))
    
    # Test 5: Chargement données CSV
    print("\n📋 Test 5: Chargement données CSV")
    try:
        patients = charger_donnees_csv("data/patients.csv")
        assert len(patients) > 0, "Aucun patient chargé"
        assert all(hasattr(p, 'id') for p in patients), "Attributs patients invalides"
        print(f"   ✅ Chargement CSV: RÉUSSI ({len(patients)} patients)")
        tests_resultats['passes'] += 1
        tests_resultats['total'] += 1
    except Exception as e:
        print(f"   ❌ Chargement CSV: ÉCHEC - {str(e)}")
        tests_resultats['echecs'] += 1
        tests_resultats['total'] += 1
        tests_resultats['details'].append(('CSV Loading', str(e)))
    
    # Résumé final
    print("\n" + "="*60)
    print("📊 RÉSULTATS DES TESTS")
    print("="*60)
    print(f"Total de tests: {tests_resultats['total']}")
    print(f"✅ Réussis: {tests_resultats['passes']}")
    print(f"❌ Échecs: {tests_resultats['echecs']}")
    print(f"📈 Taux de succès: {(tests_resultats['passes']/tests_resultats['total']*100):.1f}%")
    
    if tests_resultats['details']:
        print("\n⚠️ Détails des échecs:")
        for test_nom, erreur in tests_resultats['details']:
            print(f"   - {test_nom}: {erreur}")
    
    print("\n" + "="*60)
    
    return tests_resultats

# Note: La fonction tests_automatises() sera exécutée manuellement par l'utilisateur
print("\n💡 Pour exécuter la suite de tests, appelez: tests_automatises()")


💡 Pour exécuter la suite de tests, appelez: tests_automatises()


### 📊 Analyse Comparative et Conclusion

#### Analyse Comparative des 4 Approches

| Approche | Forces | Faiblesses | Cas Idéaux |
|---------|--------|------------|-------------|
| Agent Diagnostic | Rapidité, Interprétabilité | Dépendance règles, Non-adaptatif | Diagnostic initial, triage |
| Algorithme A* | Optimalité, Exploration systématique | Complexité exponentielle, Dépendance heuristique | Diagnostic complexe, espace défini |
| Algorithmes Génétiques | Optimisation globale, Robustesse | Pas d'optimalité, Réglages délicats | Optimisation paramètres, adaptation |
| Solveur Z3 | Garantie contraintes, Efficacité | Complexité théorique, Modélisation difficile | Validation protocoles, cohérence |

#### Recommandations d'Amélioration

1. **Intégration hybride** : Combiner les forces de chaque approche
2. **Apprentissage automatique** : Optimiser les heuristiques et paramètres
3. **Interface clinicien** : Rendre le système utilisable en pratique
4. **Validation clinique** : Tester sur des cohortes réelles

### 🎯 Conclusion

Ce notebook vous a permis d'implémenter un système de diagnostic médical multi-approches combinant :

✅ **Intelligence Artificielle Exploratoire** : Recherche (A*), heuristiques
✅ **Intelligence Artificielle Symbolique** : Règles, contraintes (Z3)
✅ **Optimisation** : Algorithmes génétiques
✅ **Application Biomédicale** : Diagnostic du diabète de type 2

Les compétences développées sont directement transférables à d'autres domaines nécessitant des systèmes de décision intelligents.